<h1>Bisection Method</h1>

<blockquote><p>The Intermediate Value Theorem: Let $f(x)$ be a <em>continuous</em> function on $[a,b]$ with the property that $f(a)$ and $f(b)$ have a different sign ($[a,b]$ is a bracket). Then there exists a value $c$ in $[a,b]$ with $f(c) = 0$. </p></blockquote>

<p>This gives us a guarantee that under some assumptions the equation $f(x) = 0$ has a solution in the interval.</p>

<p>But it does not tell us what a value of $c$ is.</p>

<h2>A recipe to find $c$</h2>

<p>Consider the following algorithm starting with $[a,b]$ and $f$ as in the IVT and, say, $f(a) < 0$.</p>

<ul><li>compute the midpoint $c = (a+b)/2$</li></ul>

<ul><li>compute $f(c)$</li></ul>

<p>––</p>

<p>- If $f(c) == 0$ we are done</p>

<p>- If $f(c) < 0$ then our <em>new</em> interval is $[c,b]$</p>

<p>- If $f(c) > 0$ our <em>new</em> interval is $[a,c]$</p>

<p><a href="http://calculuswithjulia.github.io/limits/intermediate_value_theorem.html#Bolzanoandthebisectionmethod">Repeat</a>.</p>

<p>What does this algorithm produce? At each step we get a value of $c$. To distinguish, call them $c_i$. Then we have two possibilities:</p>

<p>- $f(c_i) = 0$ for some $i$ and we are done</p>

<p>- $f(c_i) \neq 0$ for all $i$.</p>

<p>In the latter case, what to do?</p>

<p>Let $c = \lim_i c_i$. Then:</p>

<p>- $c$ exists! - $f(c) = 0$.</p>

<p>Why? Split the values $c_i$ into those that have $f(c_i) < 0$ and those that have $f(c_i) > 0$. Call these sequences $l_i$ and $u_i$.</p>

<p>If a sequence is infinite, it must converge to $c$. If both are infinite, then one limit is less than or equal to $0$, the other greater than or equal to $0$, so must be $0$.</p>

<p>If a sub-sequence is finite, say terminating at $N$. The $c_i, i \geq N$ has $f(c_i)$ all of the same sign. So in the choice which endpoint to replace $[a,b]$ one is always chosen. Say $f(a) > 0$ and $f(c_i) < 0$ for all $i > N$. It must be that $c_i$ converges to $a$, so we have $f(c_i)$ converges to $f(a)$ which is positive, but each $f(c_i)$ is negative. So this can't happen.</p>

<h2>Bisection method in floating point</h2>

<p>The above is true mathematically, but can it be true in floating point?</p>

<p>- The $c_i$ in any algorithm are necessarily finite in number, so they can't take on any value, just machine numbers. Since machine numbers are rational numbers in binary, numbers like $\sqrt{2}$ can not be represented exactly, so the value of $c$ is not guaranteed!</p>

<p>- Even the simple statement $c = (a+b)/2$ has problems! $x+y$ can overflow, It isn't even guaranteed that $fl((x+y)/2)$ is in $[x,y]$ with some rounding modes!</p>

<h3>What to do</h3>

<p>Use thresholds:</p>

<p>Stop the algorithm if</p>

<p>- $|f(c_i)| < \epsilon$</p>

<p>- $|b_i - a_i| < \delta$.</p>

<h2>An algorithm (take 1)</h2>

In [ ]:
function bisectme(f, a, b)
    delta, epsilon=4eps(), 1e-12
	@assert f(a) * f(b) < 0  # check bracket
    c = (a + b)/2
	while abs(f(c)) >= epsilon && (b-a) >= delta
	  c = (a + b) / 2
	  f(c) == 0.0 && return(c)
	  f(c)*f(a) < 0 ? (b = c) : (a = c)
	end
	return(c)
end

bisectme (generic function with 1 method)

<p>Solve $f(x) = e^x - \sin(x) = 0$ in the interval $[-4, -3]$:</p>

In [ ]:
f(x) = exp(x) - sin(x)
xstar = bisectme(f, -4, -3)
xstar, f(xstar)

(-3.1830630119329726,4.0688286073731206e-13)

<h2>How fast does this converge?</h2>

<p>Theoretically, at each step we have $a_i < c_i < b_i$. Let $c$ be a solution. Then how big is $|c - c_i|$?</p>

<p>$$ |c - c_i| \leq \frac{1}{2} (b_i - a_i) $$</p>

<p>But:</p>

<p>$$ b_i - a_i = (1/2)(b_{i-1} - a_{i-1}) = (1/2)^2 (b_{i-2} - a_{i-2}) = \cdots = 2^{i}(b_0 - a_0). $$ </p>

<p>So,</p>

<p>$$ |c - c_i| \leq \frac{1}{2^{i+1}} (b_0 - a_0). $$</p>

<p>If we have $a_0, b_0 = -4, -3$, how many steps to get $|c - c_i| < 10^{-15}$?</p>

<p>We'd need to solve for the smallest $i$ so that:</p>

<p>$$ \frac{1}{2^{i+1}} (b_0 - a_0) = \frac{1}{2^{i+1}} < 10^{-15} $$</p>

In [ ]:
ceil(log2(1e-15) - 1)

-50.0

<h2>Can we do better?</h2>

<p>Mathematically, we can always subdivide $[a,b]$ via $c=(a+b)/2$. Not so with the computer.</p>

<p>We said that using <code>c = (a+b)/2</code> has issues (overflow, loss of low bits...)</p>

<p>We avoid some of that with <code>c = a + (b-a)/2</code></p>

<p>Or we could have tried <code>c = a + b/2 - a/2</code></p>

<p>These aren't all the same:</p>

In [ ]:
a, b = prevfloat(0.0), nextfloat(0.0)
(a + b)/2, a + (b-a)/2, a + b/2 - a/2

(0.0,0.0,-5.0e-324)

In [ ]:
b = prevfloat(Inf)
a = prevfloat(prevfloat(b))
(a + b)/2, a + (b-a)/2, a + b/2 - a/2

(Inf,1.7976931348623155e308,Inf)

<p>The problem is studied in <a href="https://hal.archives-ouvertes.fr/hal-00576641v1/document">this paper</a>.</p>

<p>Without care, the following two properties of $m(I)$ are not guaranteed:</p>

<ul><li>$m(I) \in I$</li><li>for all machine numbers $v$, $|c - v| \geq |c - M(I)|$ for $c = (a + b)/2 \in R$.</li></ul>

<p>That is, we can't be sure that this will stop appropriately</p>

<pre>c = (a + b)/2
if a < c < b
   ....
end
</pre>

<h3>One solution</h3>

<p>In <code>Julia</code>'s <code>Roots</code> package they use something different by Jason <a href="http://squishythinking.com/2014/02/22/bisecting-floats/">Merrill</a></p>

<p>We can see the following:</p>

<pre>x1_int = reinterpret(Uint64, abs(x1))
x2_int = reinterpret(Uint64, abs(x2))
unsigned = reinterpret(Float64, (x1_int + x2_int) >> 1)
</pre>

<p>In practice this converts the (positive) floating point numbers into an ordered set of integers:</p>

In [ ]:
as = rand(1)
for i in 1:9
  push!(as, nextfloat(as[end]))
end
map(x -> bits(reinterpret(Uint64, x)), as)

10-element Array{ASCIIString,1}:
 "0011111111101010100000001101101001101010000000001000010001001000"
 "0011111111101010100000001101101001101010000000001000010001001001"
 "0011111111101010100000001101101001101010000000001000010001001010"
 "0011111111101010100000001101101001101010000000001000010001001011"
 "0011111111101010100000001101101001101010000000001000010001001100"
 "0011111111101010100000001101101001101010000000001000010001001101"
 "0011111111101010100000001101101001101010000000001000010001001110"
 "0011111111101010100000001101101001101010000000001000010001001111"
 "0011111111101010100000001101101001101010000000001000010001010000"
 "0011111111101010100000001101101001101010000000001000010001010001"

<p>Using this for finding the midpoint (suitably configured to handle mixed signs) means that in no more than 64 steps this process is guaranteed to converge. This means we can skip the check on the size of $f(x_n)$ to look something like:</p>

<pre>while a < midpoint(a,b) < b
  do something...
end
</pre>

<p>What happens at each step: we must have:</p>

<p>- <code>f(c) == 0</code> (exactly), or</p>

<p>- either <code>f(c) * f(a) < 0</code> or <code>f(c) * f(b) < 0</code></p>

<p>So if the answer is not exact, we guarantee that this condition holds for the returned value of <code>c</code></p>

<blockquote><p>It must be that <code>f(prevfloat(c)) * f(nextfloat(c)) <= 0</code>. </p></blockquote>

<p>(Even without IVT).</p>

In [ ]:
using Roots
f(x) = sin(x)
c = fzero(f, [3,4])
c, f(c), sign(f(prevfloat(c))) * sign(f(nextfloat(c)))

(3.141592653589793,1.2246467991473532e-16,-1.0)

In [ ]:
f(x) = 1/x
c = fzero(f, [-1, 1])
c, f(c), sign(f(prevfloat(c))) * sign(f(nextfloat(c)))

(0.0,Inf,-1.0)